References:
- https://docs.python.org/3/reference/datamodel.html
- Fluent Python by Luciano Ramalho. Chapter 12: Inheritance: For Good or For Worse

    * The pitfalls of subclassing from built-in types
    * Multiple inheriance and the method resolution order
    
#  Subclassing Built-in Types is Tricky



In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2) # duplicate values when storing for visible effect
        
# the __init__ method inherited was ignored
dd = DoppelDict(one=1)
dd

{'one': 1}

In [2]:
# [] operators calls the created __setitem__  above
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [3]:
# the update method does not use __setitem__ either
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [4]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict(a = 'Foo')
ad['a']

42

In [5]:
d = {}
d.update(ad)
d['a']

'Foo'

In [6]:
d

{'a': 'Foo'}

In [7]:
import collections

In [8]:
class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [9]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [10]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [11]:
dd.update(three = 3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [12]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [13]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [14]:
d = {}
d.update(ad)
d['a']

42

In [15]:
d

{'a': 42}

#  Multiple Inheritance and Method Resolution Order
### Diamond Problem:
Implementation of multiple inheritance dealing with potential naming conflicts when unrelated ancester classes implement a method by the same name

<img src="diamond.png" width="50%">

In [16]:
class A:
    def ping(self):
        print('ping:', self)


class B(A):
    def pong(self):
        print('pong:', self)


class C(A):
    def pong(self):
        print('PONG:', self)


class D(B, C):

    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [17]:
d = D()
d.pong()

pong: <__main__.D object at 0x111f03810>


In [18]:
C.pong(d)

PONG: <__main__.D object at 0x111f03810>


In [19]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

#### Bypass  MRO and invoke a method on a superclass directly:

In [20]:
class D(B, C):

    def ping(self):
        A.ping(self) #  instead of super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)


In [21]:
d = D()
d.ping()

ping: <__main__.D object at 0x111f16750>
post-ping: <__main__.D object at 0x111f16750>


In [22]:
d.pingpong()

ping: <__main__.D object at 0x111f16750>
post-ping: <__main__.D object at 0x111f16750>
ping: <__main__.D object at 0x111f16750>
pong: <__main__.D object at 0x111f16750>
pong: <__main__.D object at 0x111f16750>
PONG: <__main__.D object at 0x111f16750>


In [23]:
bool.__mro__

(bool, int, object)

In [24]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [25]:
print_mro(bool)

bool, int, object


In [26]:
from frenchdeck2 import FrenchDeck2

In [27]:
print_mro(FrenchDeck2)

FrenchDeck2, MutableSequence, Sequence, Reversible, Collection, Sized, Iterable, Container, object


In [28]:
import numbers

In [29]:
print_mro(numbers.Integral)

Integral, Rational, Real, Complex, Number, object


In [30]:
import io

In [31]:
print_mro(io.BytesIO)

BytesIO, _BufferedIOBase, _IOBase, object


In [32]:
print_mro(io.TextIOWrapper)

TextIOWrapper, _TextIOBase, _IOBase, object


    import tkinter
    print_mro(tkinker.Text)

```Text, Widget, BaseWeight, Misc, Pack, place, Grid, XView, YView, Object```

<img src="tkinter.png">

# Multiple Inheritance in the Real World

<img src="tkintergui.png">

# Coping with Multiple Inheritance

1. Distinguish Interface Inheritance from Implementation Inheritance
    * Inheritance of interface creates a subtype, implying a `is-a` relationship
    * Inheritance of implementation avoids code duplication by reuse
2. Make Interface Explicit with ABCs
3. Use Mixins for Code Reuse
4. Make Mixins Explicit by Naming
5. An ABC May Also Be a Mixin; The Reverse Is Not True
6. Don't Subclass from More Than One Concrete Class
7. Provide Aggregate Classees to Users
8. "Favor Object Composition Over Class Inheritance" _-Design Patterns_

<img src="django1.png">

<img src="django2.png">